In [1]:
import pandas as pd
import pyodbc
import os
t_start0 = pd.Timestamp.now()
print(t_start0)

2023-02-16 12:49:32.364754


In [2]:
def rmmk(dir): # remove exiting folder and remake folder
    import os
    import shutil

    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)

In [3]:
def printAllContents(ctx, relativeUrl):

    try:
        
        libraryRoot = ctx.web.get_folder_by_server_relative_url(relativeUrl)
        ctx.load(libraryRoot)
        ctx.execute_query()

        folders = libraryRoot.folders
        ctx.load(folders)
        ctx.execute_query()

        for myfolder in folders:
            #print("Folder name: {0}".format(myfolder.properties["Name"]))
            print("Folder name: {0}".format(myfolder.properties["ServerRelativeUrl"]))
            printAllContents(ctx, relativeUrl + '/' + myfolder.properties["Name"])
            
        files = libraryRoot.files
        ctx.load(files)
        ctx.execute_query()

        for myfile in files:
            print("File name: {0}".format(myfile.properties["ServerRelativeUrl"]))
            pathList = myfile.properties["ServerRelativeUrl"].split('/')
            fileDest = outputDir + "/"+ pathList[-1]
            downloadFile(ctx, fileDest, myfile.properties["ServerRelativeUrl"])
            
    except:
        
        print('Problem printing out list of folders')   
        sys.exit(1)  

In [4]:
def save_txt(src,dst):
    with open(os.path.join(dst), 'a', encoding ='utf8') as txt_file:
            txt_file.write(src+'\n')

In [5]:
# https://gist.github.com/hyunjong-lee/e0f97aa0e4c353c65c31b596765ee641
import os
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file_system_object_type import FileSystemObjectType
from office365.sharepoint.files.file import File
from tqdm.notebook import tqdm
import keyring

sharepoint_user = keyring.get_password("sharepointonline_username", "username")
sharepoint_pass = keyring.get_password("sharepointonline_pass","password")

baseurl = 'https://irpcplc.sharepoint.com'
basesite = '/sites/FLEQBOM' # every share point has a home.
docurl = '/Shared%20Documents/User_keydata'
siteurl = baseurl + basesite 
# target_url = basesite+docurl

ctx_auth = AuthenticationContext(siteurl)
ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_pass)
ctx = ClientContext(siteurl, ctx_auth) # make sure you auth to the siteurl.

doc_lib = ctx.web.lists.get_by_title("Documents")
items = doc_lib.items.select(["FileSystemObjectType"]).expand(["File", "Folder"]).get().execute_query()
folders = []
files = []
dst_dir = './sharepointindex'
rmmk(dst_dir)
for item in tqdm(items):  # type: ListItem
    if item.file_system_object_type == FileSystemObjectType.Folder:
        print("Folder url: {0}".format(item.folder.serverRelativeUrl))
        src_folder = item.folder.serverRelativeUrl
        save_txt(src_folder,os.path.join(dst_dir,'sharepointfolders.txt'))
        folders.append(item.folder.serverRelativeUrl)
    else:
        print("File url: {0}".format(item.file.serverRelativeUrl))
        src_file = item.file.serverRelativeUrl
        save_txt(src_file,os.path.join(dst_dir,'sharepointfiles.txt'))
        files.append(item.file.serverRelativeUrl)

  0%|          | 0/100 [00:00<?, ?it/s]

Folder url: /sites/FLEQBOM/Shared Documents/ที่มาที่ไป
Folder url: /sites/FLEQBOM/Shared Documents/SAP_connection
Folder url: /sites/FLEQBOM/Shared Documents/Exported
Folder url: /sites/FLEQBOM/Shared Documents/Form
Folder url: /sites/FLEQBOM/Shared Documents/old_4users
Folder url: /sites/FLEQBOM/Shared Documents/Plan
Folder url: /sites/FLEQBOM/Shared Documents/4users
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/-
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/11M
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/12E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/12I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13E
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13I
Folder url: /sites/FLEQBOM/Shared Documents/User_keydata/13M
Folder url: /sites/FLEQBO

In [6]:
# https://www.pythontutorial.net/python-basics/python-filter-list/
# response = File.open_binary(ctx, f"/{filtered[1].split('/',3)[-1]}")
# with open(f"./user_keydata/{filtered[1].split('/')[-1]}", "wb") as local_file:
#     local_file.write(response.content)

In [7]:
# https://www.pythontutorial.net/python-basics/python-filter-list/
rmmk('./user_keydata')
filtered = filter(lambda files: 'User_keydata' in files, files)
filtered = list(filtered)
for i,f in enumerate(filtered):
    print(i,f)
    if f.split('/')[-1][0] <= 'S':
        response = File.open_binary(ctx, f"/{f.split('/',3)[-1]}")
        with open(f"./user_keydata/{f.split('/')[-1]}", "wb") as local_file:
            local_file.write(response.content)

0 /sites/FLEQBOM/Shared Documents/User_keydata/-/-.xlsx
1 /sites/FLEQBOM/Shared Documents/User_keydata/11E/11E.xlsx
2 /sites/FLEQBOM/Shared Documents/User_keydata/11I/11I.xlsx
3 /sites/FLEQBOM/Shared Documents/User_keydata/11M/11M.xlsx
4 /sites/FLEQBOM/Shared Documents/User_keydata/12E/12E.xlsx
5 /sites/FLEQBOM/Shared Documents/User_keydata/12I/12I.xlsx
6 /sites/FLEQBOM/Shared Documents/User_keydata/13E/13E.xlsx
7 /sites/FLEQBOM/Shared Documents/User_keydata/13I/13I.xlsx
8 /sites/FLEQBOM/Shared Documents/User_keydata/13M/13M.xlsx
9 /sites/FLEQBOM/Shared Documents/User_keydata/14E/14E.xlsx
10 /sites/FLEQBOM/Shared Documents/User_keydata/14M/14M.xlsx
11 /sites/FLEQBOM/Shared Documents/User_keydata/15C/15C.xlsx
12 /sites/FLEQBOM/Shared Documents/User_keydata/21E/21E.xlsx
13 /sites/FLEQBOM/Shared Documents/User_keydata/21I/21I.xlsx
14 /sites/FLEQBOM/Shared Documents/User_keydata/21M/21M.xlsx
15 /sites/FLEQBOM/Shared Documents/User_keydata/23M/23M.xlsx
16 /sites/FLEQBOM/Shared Documents/Use